In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
from skimage.transform import SimilarityTransform
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack, save_array_as_hyperstack_tiff, \
    plane_detection_with_iterative_alignment, warp_stack_to_plane

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Choose a sample ID
sample_id = '20220511_RM0008_126hpf_fP10_f2'  # Replace with your sample ID

# Load experiment configuration
exp = sample_db.get_sample(sample_id)

# Making shortcuts of sample parameters/information
sample = exp.sample
root_path = exp.paths.root_path
trials_path = exp.paths.trials_path
anatomy_path = exp.paths.anatomy_path
n_planes = exp.params_lm.n_planes

# Define the path for the preprocessed folder
processed_folder = os.path.join(trials_path, "processed")
os.makedirs(processed_folder, exist_ok=True)

# Load planes and stack
ref_images_path = os.path.join(processed_folder, f"sum_raw_trials_{sample.id}.tif")
lm_planes = imread(ref_images_path)
lm_stack = load_tiff_as_hyperstack(os.path.join(anatomy_path, "raw", f"{sample.id}_anatomyGFRF_001_.tif"), n_channels=2)[0]

# Display the first LM plane
plt.figure(figsize=(10, 10))
plt.imshow(lm_planes[0, -1], cmap='gray')
plt.title('First LM Plane')
plt.axis('off')
plt.show()

# Process the first plane
lm_plane = lm_planes[0, -1]
final_transform, all_transformation_matrices = plane_detection_with_iterative_alignment(
    lm_plane, lm_stack, equalize=True, binning=True, plot=True, nx=2, ny=3, tiles_filter=None,
    thickness_values=None
)

# Display the matched slice
last_matched_slice = warp_stack_to_plane(lm_stack, lm_plane, SimilarityTransform(final_transform), 20)[10]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(lm_plane, cmap='gray')
ax1.set_title('Original Plane')
ax1.axis('off')
ax2.imshow(last_matched_slice, cmap='gray')
ax2.set_title('Matched Slice')
ax2.axis('off')
plt.tight_layout()
plt.show()

# Display the first tile of the LM plane
tile_size_x = lm_plane.shape[1] // 2
tile_size_y = lm_plane.shape[0] // 3
first_tile = lm_plane[:tile_size_y, :tile_size_x]

plt.figure(figsize=(10, 10))
plt.imshow(first_tile, cmap='gray')
plt.title('First Tile of LM Plane')
plt.axis('off')
plt.show()

# You can add more cells to process other planes or display intermediate results